### Imports

In [ ]:
import os
import glob
import argparse
import time
from PIL import Image
import numpy as np
import PIL
import h5py

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from tensorflow.keras.models import load_model
from layers import BilinearUpSampling2D
from loss import depth_loss_function
from utils import predict
from matplotlib import pyplot as plt

### Load Model

In [ ]:
# Argument Parser
parser = argparse.ArgumentParser(description='High Quality Monocular Depth Estimation via Transfer Learning')
parser.add_argument('--model', default='nyu.h5', type=str, help='Trained Keras model file.')
parser.add_argument('--traininput', default='training_images/*.jpg', type=str, help='Training input filename or folder.')
parser.add_argument('--testinput', default='testing_image/*.jpg', type=str, help='Testing input filename or folder.')
args, unknown = parser.parse_known_args()

# Custom object needed for inference and training
start = time.time()
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': depth_loss_function}

print('Loading model...')

# Load model into GPU / CPU
model = load_model(args.model, custom_objects=custom_objects, compile=False)

print('\nModel loaded ({0}).'.format(args.model))

### Predict Heatmap Image

In [ ]:
def load_images_with_resize(image_files):
    loaded_images = []
    for file in image_files:
        im = Image.open( file )
        im = im.resize((640, 480), PIL.Image.ANTIALIAS)
        x = np.clip(np.asarray(im, dtype=float) / 255, 0, 1)
        loaded_images.append(x)
    return np.stack(loaded_images, axis=0)

# Input images
inputs = load_images_with_resize( glob.glob(args.traininput) )
print('\nLoaded ({0}) images of size {1}.'.format(inputs.shape[0], inputs.shape[1:]))

# Compute results
outputs = predict(model, inputs)

end = time.time()
print('It took: ', end - start)

### Display Relative and True Depth

In [ ]:
def display(image, title=''):
    plt.figure(figsize=(10,5))
    out = plt.imshow(image, cmap='plasma', interpolation='nearest')
    out = plt.colorbar()
    plt.title(title)

# Delete every other row and column of the array so it matched dimensions of out predicted image
def condense_image(image_src):
    image = np.copy(image_src)
    for row in reversed(range(len(image))):
        if row % 2 != 0:
            image = np.delete(image, row, 0)

    for column in reversed(range(len(image[0]))):
        if column % 2 != 0:
            image = np.delete(image, column, 1)
    return image

In [ ]:
def get_ground_truth(index: int):
    f = h5py.File('nyu_depth_v2_labeled.mat', 'r')
    nyu_depth_image = f['depths'][index]

    return condense_image(nyu_depth_image.T)

def get_rgb(index: int):
    f = h5py.File('nyu_depth_v2_labeled.mat', 'r')
    nyu_image = f['images'][index]

    return nyu_image.T

In [ ]:
# Normal Image
rgb_image = get_rgb(0)
plt.figure(figsize=(10,5))
out = plt.imshow(rgb_image)

In [ ]:
# Display Ground-Truth Image
true_depth_image = get_ground_truth(0)
display(true_depth_image, "True Depth Image (m)")

In [ ]:
# Display Predicted Image
def normalize_image(image):
    min = np.min(image)
    max = np.max(image)
    return (image - min) / (max - min)

predicted_image = normalize_image(np.asarray(outputs[0]))
display(predicted_image, "Relative Depth Estimation")

### Calculate Function Between Relative and True Depth

In [ ]:
# Display the two images as a function between each other
x_axis = predicted_image.flatten()
y_axis = true_depth_image.flatten()

a, b = np.polyfit(x_axis, y_axis, 1)

plt.scatter(x_axis, y_axis, s=0.5)
plt.title('Relative vs True Depth')
plt.xlabel('Relative Depth Value')
plt.ylabel('True Depth Value (m)')
plt.plot(x_axis, a*x_axis + b, linewidth=2, color='red')
plt.text(0, 3.48, 'true_depth = ' + '{:.2f}'.format(b) + ' + {:.2f}'.format(a) + ' * relative_depth', size=7)
plt.show()

We should probably add more code here to have the equation based on more than one image

### Apply Equation to New Image

In [ ]:
# Apply the equation to a new image
ground_truth_image = get_ground_truth(13) # NOTE: This index must match the image in /testing_image
test_inputs = load_images_with_resize(glob.glob(args.testinput))

print('\nLoaded ({0}) images of size {1}.'.format(test_inputs.shape[0], test_inputs.shape[1:]))
test_outputs = predict(model, test_inputs)
print('Done with prediction(s)')

relative_depth_image = normalize_image(np.asarray(test_outputs[0]))
estimated_true_depth_image = a * relative_depth_image + b

display(estimated_true_depth_image, "True Depth Estimation (in meters)")
display(ground_truth_image, "Ground Truth Image (in meters)")